In [1]:
import config
import modules.dataloaders as data_loaders
#import modules.utils as utils

import matplotlib.pyplot as plt
import numpy as np
import torch
import random

import cv2

import onnx
import onnxruntime

# Save Folders

In [2]:
save_folder = './predicted_images/onnx_classify_thres_&_detect/'
save_only_detection = save_folder + 'only_detection'
save_classify_05_detect_02 = save_folder + 'classify_050_detect_2e-1'
save_classify_03_detect_02 = save_folder + 'classify_030_detect_2e-1'
# save_classify_detect_01 = save_folder + 'classify_detect_1e-1'
# save_classify_detect_001 = save_folder + 'classify_detect_1e-2'

# Datasets

Change DS_LEN in config, so list is shuffled first and VAL Datasets are more diverse. To draw 128 pictures, choose DS_LEN = 168, so it drops some overlapped and mora than x objects, but you still have 128 pictures.

RS Dataset length is about 342 pictures with MAX_OBJ = 5, so 128 is ok.

### Random Seed

Initialize it to shuffle list inside datasets always in the same order

In [3]:
random.seed(123)

In [4]:
val_loader = data_loaders.get_val_loader(
    dfire_len = 200,
    fasdd_uav_len = 200,
    fasdd_cv_len = 200)


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 20
DFire Removed due to more than 10: 0

Test DFire dataset len: 180

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 27
FASDD Removed due to more than 10: 12

Test FASDD UAV dataset len: 161

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 6
FASDD Removed due to more than 10: 2

Test FASDD CV dataset len: 192

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 341
Concatenate with FASDD CV dataset
Test dataset len: 533


# Check ONNX Models

In [5]:
classifier = onnx.load('./onnx_models/medium_fassd__conv341_big__epoch=93.onnx')
onnx.checker.check_model(classifier)

detector = onnx.load('./onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx')
onnx.checker.check_model(detector)

# Functions to Plot Predictions

In [6]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [7]:
def iou(
    boxes_preds, boxes_labels, 
    box_format="midpoint",
    epsilon=1e-6
):
    """
    Calculates intersection over union for bounding boxes.
    
    :param boxes_preds (tensor): Bounding box predictions of shape (BATCH_SIZE, 4)
    :param boxes_labels (tensor): Ground truth bounding box of shape (BATCH_SIZE, 4)
    :param box_format (str): midpoint/corners, if boxes (x,y,w,h) format or (x1,y1,x2,y2) format
    :param epsilon: Small value to prevent division by zero.
    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == 'midpoint':
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    if box_format == 'corners':
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4] 
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    union = (box1_area + box2_area - intersection + epsilon)

    iou = intersection / union
    #print(f'IOU is numpy: {iou.numpy()}')

    return iou

def non_max_supression(bboxes, 
                       iou_threshold=config.IOU_THRESHOLD, 
                       score_threshold=config.SCORE_THRESHOLD, 
                       box_format="corners"):
    """
    Does Non Max Suppression given bboxes

    Parameters:
        bboxes (list): list of lists containing all bboxes with each bboxes
        specified as [x1, y1, x2, y2, confidence, class_id] MY FORMAT VERSION       
        iou_threshold (float): threshold where predicted bboxes is correct
        score_threshold (float): threshold to remove predicted bboxes (independent of IoU) 
        box_format (str): "midpoint" or "corners" used to specify bboxes

    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    """

    assert type(bboxes) == list

    bboxes = [box for box in bboxes if box[4] > score_threshold]
    bboxes = sorted(bboxes, key=lambda x: x[4], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[5] != chosen_box[5]
            or iou(
                torch.tensor(chosen_box[:4]),
                torch.tensor(box[:4]),
                box_format=box_format,
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms


# ______________________________________________________________ #
# ____________________      Pred Boxes      ____________________ #
# ______________________________________________________________ #
def get_best_box_and_class(out):
    
    conf_1 = out[..., 4:5]
    conf_2 = out[..., 9:10]
    confs = torch.cat((conf_1, conf_2), dim=-1)
    _, idx = torch.max(confs, keepdim=True, dim=-1)
    
    best_boxes = idx*out[..., 5:10] + (1-idx)*out[..., 0:5]
    
    _, class_idx = torch.max(out[..., 10:12], keepdim=True, dim=-1)
    
    best_out = torch.cat((best_boxes, class_idx), dim=-1)

    return best_out

def get_bboxes_from_model_out(model_out, 
                              iou_threshold=config.IOU_THRESHOLD, 
                              score_threshold=config.SCORE_THRESHOLD):

    model_out = get_best_box_and_class(model_out.detach().to('cpu'))
    
    c2b_mtx = np.zeros((config.S, config.S, 2))
    for j in range(config.S):
        for i in range(config.S):
            c2b_mtx[i, j, 0] = j
            c2b_mtx[i, j, 1] = i

    model_out = model_out.numpy()
    out_xy = model_out[..., :2]
    out_rest = model_out[..., 2:]

    c2b_xy = (c2b_mtx+out_xy)/config.S
    out = np.concatenate((c2b_xy, out_rest), axis=-1)
    #print(f'Concat out\n {out}')

    bboxes_flat = np.reshape(out, (config.S*config.S, 5+1)) # Replace 5+C by 5+1, as we filtered best class before (get_best_box_and_class)
    bboxes_list = [bbox for bbox in bboxes_flat.tolist()]

    nms_pred_bboxes = non_max_supression(
        bboxes_list,
        iou_threshold=iou_threshold, 
        score_threshold=score_threshold, 
        box_format="midpoint")

    return nms_pred_bboxes


# ______________________________________________________________ #
# ____________________      True Boxes      ____________________ #
# ______________________________________________________________ #
def get_bboxes_from_label_mtx(label_mtx):
    '''
    Receives a label_mtx, as yielded by dataset or dataloader and returns a list of bounding boxes.
    
    Arguments:
        - label_mtx
    
    Returns:
        - bboxes_list: list with all cells containing score = 1
            [xcell, ycell, w, h, score, smoke, fire] -> [x, y, w, h, 1, smoke, fire]
    '''

    c2b_mtx = np.zeros((config.S, config.S, 2))
    for j in range(config.S):
        for i in range(config.S):
            c2b_mtx[i, j, 0] = j
            c2b_mtx[i, j, 1] = i

    label_mtx = label_mtx.numpy()
    label_xy = label_mtx[..., :2]
    label_rest = label_mtx[..., 2:]

    c2b_xy = (c2b_mtx+label_xy)/config.S
    out = np.concatenate((c2b_xy, label_rest), axis=-1)
    #print(f'Concat out\n {out}')

    bboxes_list = np.reshape(out, (config.S*config.S, 5+config.C))

    bboxes_list = [bbox for bbox in bboxes_list.tolist() if bbox[4]==1]

    return bboxes_list


# ______________________________________________________________ #
# ____________________        Plots         ____________________ #
# ____________________ True & Pred Boxes    ____________________ #
# ______________________________________________________________ #
def plot_grid(img):
    '''
    Plot grid on top of the picture
    '''
      
    cell_size = int(config.IMG_W / config.S)
    
    # Draw horizontal lines
    for i in range(1, config.S):
        cv2.line(img, (0, cell_size*i), (config.IMG_W-1, cell_size*i), config.GRID_COLOR, 1)
    # Draw vertical lines
    for j in range(1, config.S):
        cv2.line(img, (cell_size*j, 0), (cell_size*j, config.IMG_H-1), config.GRID_COLOR, 1)
        
    return img

    
def plot_dataset_img(img, label_mtx, grid=False):
    '''
    It draws the bounding boxes over the image.

    Arguments:
        - ori_img: original image with no modification or letterbox
        - label_mtx: [xcell, ycell, w, h, score=1, smoke, fire], tensor (7, 7, 12)
        - grid: plot grid over the image

    Returns:
        - pic: picture with bounding boxes on top of original picture
    '''

    # NEVER remove copy() or use np.ascontiguousarray()
    # Convert pytorch tensor to numpy
    img = img.permute(1, 2, 0) * 256
    img = img.numpy().astype(np.uint8).copy()   
       
    if grid == True:
        img = plot_grid(img)
    
    bboxes = get_bboxes_from_label_mtx(label_mtx)

    for i,(xc, yc, w, h, score, smoke, fire) in enumerate(bboxes):
        xmin, ymin, xmax, ymax = xc - w/2, yc - h/2, xc + w/2, yc + h/2
        box = np.array([xmin, ymin, xmax, ymax]).astype(np.float32)
        box[0] = box[0]*config.IMG_W
        np.clip(box[0], 0, None)
        box[1] = box[1]*config.IMG_H
        np.clip(box[1], 0, None)
        box[2] = box[2]*config.IMG_W - 1 # avoid out of limits due to rounding
        box[3] = box[3]*config.IMG_H - 1 # avoid out of limits due to rounding
        box = box.round().astype(np.int32).tolist()
        #print(f'Box after conversion\n{box}')
        if smoke == 1:
            class_id = 0
        elif fire == 1:
            class_id = 1
        else:
            print("Wrong Class ID")
        name = config.CLASSES[class_id]
        color = config.BBOX_COLORS[name]
        cv2.rectangle(img, box[:2], box[2:], color, 1) 
        if box[1] < 30:
            if class_id == 0:
                cv2.rectangle(img, [box[0], box[1]+15], [box[0]+55, box[1]], color, -1) 
            else:
                cv2.rectangle(img, [box[0], box[1]+15], [box[0]+25, box[1]], color, -1) 
            cv2.putText(img,name,(box[0], box[1] + 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 0],
                        thickness=1)  # 0.5 -> font size
        else:
            if class_id == 0:
                cv2.rectangle(img, [box[0], box[1]-20], [box[0]+55, box[1]], color, -1) 
            else:
                cv2.rectangle(img, [box[0], box[1]-20], [box[0]+25, box[1]], color, -1) 
            cv2.putText(img,name,(box[0], box[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 0],
                        thickness=1)  # 0.5 -> font size

    return img


def plot_predicted_img(img, model_out, score_thres=None, grid=False):
    '''
    It draws the bounding boxes over the image.

    Arguments:
        - ori_img: original image with no modification or letterbox
        - model_out: [xcell, ycell, w, h, score, smoke, fire], tensor (7, 7, 12)
        - grid: plot grid over the image

    Returns:
        - pic: picture with bounding boxes on top of original picture
    '''

    # NEVER remove copy() or use np.ascontiguousarray()
    # Convert pytorch tensor to numpy
    img = img.permute(1, 2, 0) * 256
    img = img.numpy().astype(np.uint8).copy()   
       
    if grid == True:
        img = plot_grid(img)
    
    if score_thres is not None:
        bboxes = get_bboxes_from_model_out(
            model_out,
            score_threshold=score_thres)
    else:
        bboxes = get_bboxes_from_model_out(model_out)

    for xc, yc, w, h, score, class_id in bboxes:
        xmin, ymin, xmax, ymax = xc - w/2, yc - h/2, xc + w/2, yc + h/2
        box = np.array([xmin, ymin, xmax, ymax]).astype(np.float32)
        box[0] = box[0]*config.IMG_W
        np.clip(box[0], 0, None)
        box[1] = box[1]*config.IMG_H
        np.clip(box[1], 0, None)
        box[2] = box[2]*config.IMG_W - 1 # avoid out of limits due to rounding
        box[3] = box[3]*config.IMG_H - 1 # avoid out of limits due to rounding
        box = box.round().astype(np.int32).tolist()
        
        class_id = int(class_id)
        name = config.CLASSES[class_id]
        color = config.BBOX_COLORS[name]
        name += str(f' {score:.3f}')
        
        cv2.rectangle(img, box[:2], box[2:], color, 1) 
        if box[1] < 30:
            if class_id == 0:
                cv2.rectangle(img, [box[0], box[1]+15], [box[0]+105, box[1]], color, -1) 
            else:
                cv2.rectangle(img, [box[0], box[1]+15], [box[0]+80, box[1]], color, -1) 
            cv2.putText(img,name,(box[0], box[1] + 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 0],
                        thickness=1)  # 0.5 -> font size
        else:
            if class_id == 0:
                cv2.rectangle(img, [box[0], box[1]-20], [box[0]+105, box[1]], color, -1) 
            else:
                cv2.rectangle(img, [box[0], box[1]-20], [box[0]+80, box[1]], color, -1) 
            cv2.putText(img,name,(box[0], box[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 0],
                        thickness=1)  # 0.5 -> font size

    return img

# ______________________________________________________________ #
# _________________ ONNX Prediction Function  __________________ #
# ______________________________________________________________ #
def onnx_predict(
    img,
    classify_session,
    detect_session,
    classification_thres
):
    
    if classify_session is not None:
        classify_inputs = {classify_session.get_inputs()[0].name: to_numpy(img)}
        classification_out = classify_session.run(None, classify_inputs)
        # print(f'Smoke pred: {classification_out[0][0][0]}')
        # print(f'Fire pred: {classification_out[0][0][1]}')

        # Use Detector if Classifier predicts fire or smoke
        if ( classification_out[0][0][0] >= classification_thres
            or 
            classification_out[0][0][1] >= classification_thres ):
            detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(img)}
            out = detect_session.run(None, detect_inputs)
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
        else:
            out = torch.zeros(1,7,7,12)  
    
    else:
        detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(img)}
        out = detect_session.run(None, detect_inputs)
        out = torch.tensor(np.array(out[0]))
        out = out.permute(0, 2, 3, 1)
        
    # Remove batch dim, although it makes no difference for get_bboxes_from_model_out 
    out = out[0]   
    
    return out
        
# ______________________________________________________________ #
# ____________________         Plot         ____________________ #
# ____________________      N IMAGES        ____________________ #
# _________________ [ori1, pred1, ori2, pred2] _________________ #
# ______________________________________________________________ #
def plot_n_images_onnx(
    loader, 
    classification_model,
    detection_model,
    classification_thres,
    score_thres,
    n_imgs, 
    save_name):
    '''
    Plots 4 pictures in each row: [ori1, pred1, ori2, pred2]
    '''
    
    if classification_model is not None:
        classify_session = onnxruntime.InferenceSession(classification_model, providers=["CPUExecutionProvider"])
    else:
        classify_session = None
    detect_session = onnxruntime.InferenceSession(detection_model, providers=["CPUExecutionProvider"])
    
    n_imgs = int(n_imgs / 2) * 2 # Make n_imgs an even number
    rows = int(n_imgs/2)
    cols = 4
    plot_img_height = n_imgs
    
    fig, ax = plt.subplots(rows, cols, figsize=(10, n_imgs)) # 32 pics -> (10, 40) / 64 
    
    img_idx = 0
    
    for (img, label) in loader:
        
        half_batch = int(config.BATCH_SIZE/2)
        for i in range(half_batch): # index for Half Batch_Size
            
            #print(f'Batch Index: {i}')
            ax_idx = int(img_idx/4)
            #print(f'AX idx: {ax_idx}')
            #print(f'Img idx before update: {img_idx}')
    
            # Left Half: Original, Predicted
            plt.subplot(rows, cols, img_idx+1)
            if img_idx == 0:
                ax[ax_idx][0].set_title("Original") # Set title for colum 0

            ori_pic_1 = plot_dataset_img(img[2*i], label[2*i], grid=False)
            ax[ax_idx][0].imshow(ori_pic_1)
            ax[ax_idx][0].set_axis_off()

            plt.subplot(rows, cols, img_idx+2)
            if img_idx == 0:
                ax[ax_idx][1].set_title("Predicted") # Set title for colum 1

            img_to_model_1 = img[2*i].unsqueeze(dim=0) #.to(config.DEVICE)
            # pred_out_1 = model(img_to_model_1)
            # pred_out_1 = pred_out_1.permute(0, 2, 3, 1)
            
            pred_out_1 = onnx_predict(
                img_to_model_1,
                classify_session,
                detect_session,
                classification_thres=classification_thres
            )
                
            pred_pic_1 = plot_predicted_img(
                img[2*i], 
                pred_out_1, 
                score_thres=score_thres,
                grid=False)
            
            ax[ax_idx][1].imshow(pred_pic_1)
            ax[ax_idx][1].set_axis_off()

            # Right Half: Original, Predicted
            plt.subplot(rows, cols, img_idx+3)
            if img_idx == 0:
                ax[ax_idx][2].set_title("Original") # Set title for colum 2

            ori_pic_2 = plot_dataset_img(img[2*i+1], label[2*i+1], grid=False)
            ax[ax_idx][2].imshow(ori_pic_2)
            ax[ax_idx][2].set_axis_off()

            plt.subplot(rows, cols, img_idx+4)
            if img_idx == 0:
                ax[ax_idx][3].set_title("Predicted") # Set title for colum 3

            img_to_model_2 = img[2*i+1].unsqueeze(dim=0) #.to(config.DEVICE)
            # pred_out_2 = model(img_to_model_2)
            # pred_out_2 = pred_out_2.permute(0, 2, 3, 1)
            
            pred_out_2 = onnx_predict(
                img_to_model_2,
                classify_session,
                detect_session,
                classification_thres=classification_thres
            )
                
            pred_pic_2 = plot_predicted_img(
                img[2*i+1], 
                pred_out_2, 
                score_thres=score_thres,
                grid=False)

            ax[ax_idx][3].imshow(pred_pic_2)
            ax[ax_idx][3].set_axis_off()

            img_idx += 4 # Move to next row
            #print(f'Img idx after update: {img_idx}')
            
            if int(img_idx/2) == n_imgs:
                #print("Break Inner Loop")
                break
        
        if int(img_idx/2) == n_imgs:
            #print("Break Outer Loop: data loader")
            break
    
    plt.tight_layout(pad=0.5)

    if save_name is not None:
        plt.savefig(save_name + '.png')
        plt.close()
    else:
        plt.show()    




# Plot Predictions

In [8]:
N_IMGS_PLOT = 128

### Only Detection

In [9]:
plot_n_images_onnx(
    loader = val_loader, 
    classification_model =None,
    detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
    classification_thres = 0, # No impact in this case
    score_thres = 0.2,
    n_imgs = N_IMGS_PLOT,
    save_name = save_only_detection)

Traceback (most recent call last):
  File "/opt/conda/envs/onnx_eval/lib/python3.10/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/opt/conda/envs/onnx_eval/lib/python3.10/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/opt/conda/envs/onnx_eval/lib/python3.10/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/opt/conda/envs/onnx_eval/lib/python3.10/shutil.py", line 731, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/opt/conda/envs/onnx_eval/lib/python3.10/shutil.py", line 729, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-u1iba46b'


### Classify 1st + Detect 2nd: Classification Thres (50 %) = 0, Score Threshold = 0.2

In [10]:
plot_n_images_onnx(
    loader = val_loader, 
    classification_model ='./onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
    classification_thres = 0,
    score_thres = 0.2,
    n_imgs = N_IMGS_PLOT,
    save_name = save_classify_05_detect_02)

### Classify 1st + Detect 2nd: Classification Thres (30 %) = -0.8472978604, Score Threshold = 0.2

In [11]:
plot_n_images_onnx(
    loader = val_loader, 
    classification_model ='./onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
    classification_thres = -0.8472978604,
    score_thres = 0.2,
    n_imgs = N_IMGS_PLOT,
    save_name = save_classify_03_detect_02)

### Classify 1st + Detect 2nd: Score Threshold = 0.15

In [12]:
# plot_n_images_onnx(
#     loader = val_loader, 
#     classification_model ='./onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
#     detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
#     score_thres = 0.15,
#     n_imgs = N_IMGS_PLOT,
#     save_name = save_classify_detect_015)

### Classify 1st + Detect 2nd: Score Threshold = 0.1

In [13]:
# plot_n_images_onnx(
#     loader = val_loader, 
#     classification_model ='./onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
#     detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
#     score_thres = 0.1,
#     n_imgs = N_IMGS_PLOT,
#     save_name = save_classify_detect_01)

### Classify 1st + Detect 2nd: Score Threshold = 0.01

In [14]:
# plot_n_images_onnx(
#     loader = val_loader, 
#     classification_model ='./onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
#     detection_model = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx', 
#     score_thres = 0.01,
#     n_imgs = N_IMGS_PLOT,
#     save_name = save_classify_detect_001)